In [1]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.4 MB 39.2 MB/s 
     |████████████████████████████████| 4.3 MB 34.1 MB/s 
     |████████████████████████████████| 260 kB 62.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641946 sha256=7d505f312e71927ef981878ca8b3f9b96a8bac70e828388267d430ec1b9154cd
  Stored in directory: /root/.cache/pip/wheels/26/57/ce/ca63ad74b90273f9a682028d187645a42dce5c5255228d46c8
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=0622edfad86d60028f6e89ac96c0b236cf4f7c5d9fb5512a63b163bf50a9d827
  Sto

In [1]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   259  100   259    0     0   1992      0 --:--:-- --:--:-- --:--:--  1992
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [2]:
import sklearn

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/telecom_users_clean.csv')

In [7]:
df=df.drop(columns=['customerID'])

In [12]:
X = df.iloc[:,:-1]
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=19, stratify=y)

In [9]:
import autosklearn.classification
import autosklearn.metrics

In [10]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=30,
    memory_limit=None,
    seed=1,
    resampling_strategy='holdout',
    resampling_strategy_arguments={
        'train_size': 0.8,
        'shuffle': True
    },
    metric=autosklearn.metrics.f1_weighted,
    scoring_functions=[autosklearn.metrics.f1_weighted, autosklearn.metrics.balanced_accuracy,
                       autosklearn.metrics.precision_weighted, autosklearn.metrics.recall_weighted],
    tmp_folder='./autosklearn_classification_example_out',
)

In [14]:
automl.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:322: UserWarning: Input Column gender has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:322: UserWarning: Input Column SeniorCitizen has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:322: UserWarning: Input Column Partner has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:322: UserWarning: Input Column Dependents has generic type object. Autosklearn will treat this column as string. Please ensure that thi

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      memory_limit=None, metric=f1_weighted,
                      per_run_time_limit=30,
                      resampling_strategy_arguments={'shuffle': True,
                                                     'train_size': 0.8},
                      scoring_functions=[f1_weighted, balanced_accuracy,
                                         precision_weighted, recall_weighted],
                      time_left_for_this_task=300,
                      tmp_folder='./autosklearn_classification_example_out')

In [17]:
def get_metric_result(cv_results):
    results = pd.DataFrame.from_dict(cv_results)
    results = results[results['status'] == "Success"]
    cols = ['rank_test_scores', 'param_classifier:__choice__', 'mean_test_score']
    cols.extend([key for key in cv_results.keys() if key.startswith('metric_')])
    return results[cols]


print("Metric results:")
print(get_metric_result(automl.cv_results_).to_string(index=False))

Metric results:
 rank_test_scores param_classifier:__choice__  mean_test_score  metric_f1_weighted  metric_balanced_accuracy  metric_precision_weighted  metric_recall_weighted
                6               random_forest         0.728777            0.728777                  0.626476                   0.723185                0.743215
                3           gradient_boosting         0.750072            0.750072                  0.657972                   0.745674                0.759916
                4               random_forest         0.743076            0.743076                  0.640905                   0.739384                0.758873
                2           gradient_boosting         0.759414            0.759414                  0.658560                   0.758317                0.775574
                5           gradient_boosting         0.738205            0.738205                  0.640256                   0.733170                0.750522
               10       

In [15]:
models = automl.get_models_with_weights()

weights = [row[0] for row in models]
models = [row[1] for row in models]
models = [model.config.get_dictionary()['classifier:__choice__'] for model in models]

final_model = pd.DataFrame({
    'weight': weights,
    'model': models
})

final_model

,weight,model
0,0.48,gradient_boosting
1,0.24,sgd
2,0.12,gradient_boosting
3,0.10,gradient_boosting
4,0.04,libsvm_svc
5,0.02,gradient_boosting


In [16]:
models = automl.get_models_with_weights()
[row[1] for row in models][0].config.get_dictionary()

{'balancing:strategy': 'none',
 'classifier:__choice__': 'gradient_boosting',
 'data_preprocessor:__choice__': 'feature_type',
 'feature_preprocessor:__choice__': 'feature_agglomeration',
 'classifier:gradient_boosting:early_stop': 'valid',
 'classifier:gradient_boosting:l2_regularization': 1.7108930238344161e-10,
 'classifier:gradient_boosting:learning_rate': 0.010827728124541558,
 'classifier:gradient_boosting:loss': 'auto',
 'classifier:gradient_boosting:max_bins': 255,
 'classifier:gradient_boosting:max_depth': 'None',
 'classifier:gradient_boosting:max_leaf_nodes': 25,
 'classifier:gradient_boosting:min_samples_leaf': 4,
 'classifier:gradient_boosting:scoring': 'loss',
 'classifier:gradient_boosting:tol': 1e-07,
 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'mean',
 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'none',
 'data_preprocessor:feature_type:text_transformer:text_encoding:__choice__': 'tfidf_encoding',
 'data_